[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NaiaraSPinto/VegMapper/blob/master/gedi/gedi_l2a_arset.ipynb)


# 1- Install packages and import dependencies
Sets up the environment by installing and importing the required Python libraries. This includes packages for interacting with NASA data services (`earthaccess`, `harmony-py`) and for general data handling and processing.

In [ ]:
!pip install -q geoviews earthaccess harmony-py

In [ ]:
from harmony import BBox, Client, Collection, Request, CapabilitiesRequest
from datetime import datetime
import json
import earthaccess
import geopandas as gp
import os
from IPython.display import JSON
import h5py
import pandas as pd
from shapely.geometry import Point
import ast
import subprocess
import requests
import tempfile

# 2- Authenticate with your NASA Earth Data credentials

If you don't have an account, you can [register on the Earthdata website](https://urs.earthdata.nasa.gov/).

In [ ]:
auth = earthaccess.login(persist=True)

# 3- Define AOI & Output Folder

* Provide the path to a **GeoJSON file** to define your study area.
* Specify the **output folder** where your results will be saved.

In [ ]:
my_site = input("Enter the path to your GeoJSON polygon file: ")

try:
    with open(my_site) as f:
        geojson_polygon = json.load(f)
    print("GeoJSON loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {my_site}")
except json.JSONDecodeError:
    print(f"Invalid GeoJSON format in file: {my_site}")

output_folder = input("Enter the output folder where you wish to save your results:")

# 4- Define the temporal range of your search
GEDI data availability:
*   April 2019 - March 2023
*   April 2024 - Present


In [ ]:
start_str = input("Enter start date (YYYY-MM-DD): ")
stop_str = input("Enter stop date (YYYY-MM-DD): ")

# Convert input strings to datetime objects
try:
    temporal_range = {
        'start': datetime.strptime(start_str, "%Y-%m-%d"),
        'stop': datetime.strptime(stop_str, "%Y-%m-%d")
    }
    print("Temporal range set to:", temporal_range)
except ValueError:
    print("Invalid date format. Please use YYYY-MM-DD.")

# 5- Call the Harmony Service to subset GEDI granules
Submits a request to the Harmony API for server-side subsetting of the GEDI L2A collection based on the specified AOI and time range.

In [ ]:
harmony_client = Client(auth=(auth.username, auth.password))
capabilities_request = CapabilitiesRequest(short_name='GEDI02_A')
capabilities = harmony_client.submit(capabilities_request)
concept_id = capabilities['conceptId']

request = Request(
    collection = Collection(id=concept_id),
    shape = my_site,
    temporal = temporal_range
)
task = harmony_client.submit(request)
print(f'Harmony request ID: {task}')
print(f'Processing your Harmony request:')
task_json = harmony_client.result_json(task, show_progress=True)

# 6- Save URLs of subsetted granules
This step saves a list that contains the remote location of files containing GEDI shots in H5 format. Here, we are only retrieving a list of file locations (URLs), not the files themselves.

In [ ]:
h5_files = [link['href'] for link in task_json['links'] if link['href'].endswith('.h5')]
h5_file_name = 'h5_files.txt'
with open(os.path.join(output_folder, h5_file_name), 'w') as f:
    for h5_file in h5_files:
        f.write(h5_file + '\n')

print(f"H5 file links saved to {h5_file_name}")

# 7- Define Data Quality Filters
Here are some commonly-used criteria:
*   **Sensitivity** is the the maximum canopy cover through which the LiDAR can detect the ground with 90% probability, and it is useful in areas of **dense vegetation**
*   **Min RH95** can be used to **exclude bare ground**
*   **Night Time Shots** can be selected to maximize **signal-to-noise ratio**

In [ ]:
#sensitivity
while True:
    min_sens_input = input("Enter minimum sensitivity value (0.0-0.9): ")
    try:
        min_sens = float(min_sens_input)
        # Check if the float is within the desired range
        if 0.0 <= min_sens <= 0.9:
            print(f"Minimum sensitivity set to: {min_sens}")
            break  # Exit the loop if input is valid
        else:
            print("Error: The value must be between 0.0 and 0.9.")
    except ValueError:
        print("Error: Invalid input. Please enter a numerical value.")

#min RH95
while True:
    min_height_input = input("Enter minimum RH95 value (0-5): ")
    try:
        min_height = int(min_height_input)
        # Check if the integer is within the desired range
        if 0 <= min_height <= 5:
            print(f"Minimum height set to: {min_height}")
            break # Exit the loop if input is valid
        else:
            print("Error: The value must be between 0 and 5.")
    except ValueError:
        print("Error: Invalid input. Please enter a numerical value.")

#night time only
try:
    night_only = ast.literal_eval(input("Enter True or False for night only filtering: "))
    if isinstance(night_only, bool):
        print(f"The boolean value is: {night_only}")
    else:
        print("Invalid input. The input must be 'True' or 'False'.")
except (ValueError, SyntaxError):
    print("Invalid input. The input must be 'True' or 'False'.")




# 8- Function to filter GEDI L2A shots
For each H5 file, download from the DAAC, apply the user-specified filtering, and save results as a local CSV.

In [ ]:
#This is the main function
def extract_gedi_rh_metrics_from_urls(file_list_txt, csv_file, beams=None,
                                      min_sensitivity=0.9, min_rh95=0,
                                      night=False):
    """
    Reads a list of HTTPS GEDI HDF5 file URLs, downloads each file completely,
    extracts RH metrics for specified beams, filters by sensitivity, RH95,
    and optionally by solar elevation (nighttime), and saves all shots to a CSV.

    Parameters
    ----------
    file_list_txt : str
        Path to a text file containing one HTTPS URL per line.
    csv_file : str
        Output CSV file path.
    beams : list, optional
        List of beam names to extract. Defaults to the four full-power beams.
    min_sensitivity : float, optional
        Minimum acceptable sensitivity (default: 0.95)
    min_rh95 : float, optional
        Minimum acceptable RH95 value (default: 0)
    night : bool, optional
        If True, select only shots where solar_elevation < 0 (nighttime)
    """
    if beams is None:
        beams = ['BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']

    # Read list of URLs
    with open(file_list_txt, 'r') as f:
        h5_urls = [line.strip() for line in f if line.strip()]

    records = []

    for url in h5_urls:
        print(f"\n📥 Downloading {url} ...")
        try:
            response = requests.get(url, timeout=300)
            response.raise_for_status()

            with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
                tmp.write(response.content)
                tmp_path = tmp.name

            with h5py.File(tmp_path, 'r') as f:
                for beam_name in beams:
                    if beam_name not in f:
                        print(f"  ⚠️ Beam {beam_name} not found in {url}")
                        continue

                    beam = f[beam_name]

                    # Check datasets required for GEDI L2A v2.1+
                    required = ['lat_lowestmode', 'lon_lowestmode', 'rh', 'shot_number', 'solar_elevation']
                    if not all(k in beam for k in required):
                        print(f"  ⚠️ Missing required datasets in {beam_name}")
                        continue

                    # Try to access sensitivity safely
                    sensitivity = None
                    if 'QA' in beam and 'sensitivity' in beam['QA']:
                        sensitivity = beam['QA/sensitivity'][:]
                    elif 'sensitivity' in beam:
                        sensitivity = beam['sensitivity'][:]
                    else:
                        print(f"  ⚠️ Sensitivity missing in {beam_name}, skipping beam")
                        continue

                    lat = beam['lat_lowestmode'][:]
                    lon = beam['lon_lowestmode'][:]
                    shot_num = beam['shot_number'][:]
                    rh = beam['rh'][:]
                    solar = beam['solar_elevation'][:]

                    # Extract selected RH metrics
                    rh25 = rh[:, 25]
                    rh50 = rh[:, 50]
                    rh75 = rh[:, 75]
                    rh95 = rh[:, 95]

                    # Apply filters
                    mask = (sensitivity >= min_sensitivity) & (rh95 >= min_rh95)
                    if night:
                        mask &= (solar < 0)

                    if not mask.any():
                        print(f"  ⚙️ No shots passed filters in {beam_name}")
                        continue

                    df = pd.DataFrame({
                        "source_url": url,
                        "beam": beam_name,
                        "shot_number": shot_num[mask],
                        "latitude": lat[mask],
                        "longitude": lon[mask],
                        "rh25": rh25[mask],
                        "rh50": rh50[mask],
                        "rh75": rh75[mask],
                        "rh95": rh95[mask],
                        "sensitivity": sensitivity[mask],
                        "solar_elevation": solar[mask]
                    })

                    records.append(df)

        except requests.exceptions.RequestException as e:
            print(f"❌ Download failed for {url}: {e}")
        except Exception as e:
            print(f"❌ Error processing {url}: {e}")
        finally:
            if 'tmp_path' in locals() and os.path.exists(tmp_path):
                os.remove(tmp_path)

    # Combine all dataframes
    if records:
        shots_df = pd.concat(records, ignore_index=True)
        shots_df.to_csv(csv_file, index=False)
        print(f"\n✅ Total shots saved: {len(shots_df):,}")
        print(f"✅ Output file: {csv_file}")
    else:
        print("\n⚠️ No valid shots extracted.")


# 9- Run the filter function
Results will be saved in a CSV file that can be downloaded into your computer

In [ ]:
# Path to the list of GEDI L2A granules (H5 files) generated by the Harmony subset step
gedi_file = os.path.join(output_folder, h5_file_name)

out_csv_file_nm = "gedi_shots_aoi.csv"
out_csv_file = os.path.join(output_folder, out_csv_file_nm)

#List of Full Power Beams
full_power_beams = ['BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']

#Get the filtering variables
min_sens_input = float(min_sens_input)
min_height_input = int(min_height_input)
night_only = bool(night_only)

#Run the function to filter H5 file and extract RH metrics
extract_gedi_rh_metrics_from_urls(gedi_file, out_csv_file, beams=full_power_beams,
                                      min_sensitivity=min_sens_input, min_rh95=min_height_input,
                                      night=night_only)
